In [1]:
import json
import urllib2
import numpy as np
import cStringIO as StringIO
import codecs
import nltk
import re

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
source_url = r'https://data.seattle.gov/api/views/mags-97de/rows.json?accessType=DOWNLOAD'

In [3]:
response = urllib2.urlopen(source_url)

In [4]:
source = response.read()

In [5]:
full_data = json.loads(source)

In [6]:
meta = full_data['meta']
data = full_data['data']

In [7]:
id_name = 'id'
description_name = 'description'
category_name = 'category'

names = [m['name'].lower() for m in meta['view']['columns']]

In [8]:
for id_idx, name in enumerate(names):
    if name == id_name:
        break

In [9]:
for description_idx, name in enumerate(names):
    if name == description_name:
        break

In [10]:
for category_idx, name in enumerate(names):
    if name == category_name:
        break

In [11]:
def category2num(category):
    if category == 'SINGLE FAMILY / DUPLEX':
        return 1
    elif category == 'MULTIFAMILY':
        return 2
    return 0

id_list = []
description_list = []
cat_list = []

for d in data:
    if d is not None:
        id_list.append(d[id_idx])
        description_list.append(d[description_idx])
        cat_list.append(category2num(d[category_idx]))

In [13]:
print data[0][category_idx]
category2num(data[0][category_idx])

SINGLE FAMILY / DUPLEX


1

In [41]:
pat = re.compile('\D*\d')
def match_number(x):
    if re.match(pat, x) is None:
        return True

In [42]:
sample = 'jahjs a626532ggf y3 uyuy 23yy'
sample_list = sample.split(' ')
filter(match_number, sample_list)

['jahjs', 'uyuy']

In [43]:
text_list = []
indices = []
stemmer = SnowballStemmer("english", ignore_stopwords=True)

for i, d in enumerate(description_list):
    try:
        words = d.lower().split(' ')
        update = filter(match_number, words)
        text = ' '.join([stemmer.stem(word) for word in update])
        text = ' '.join(filter(match_number, text.split(' ')))
        text_list.append(text)
        indices.append(i)
    except:
        pass
identity_list = list(np.array(id_list)[indices])
category_list = list(np.array(cat_list, dtype=np.int32)[indices])

In [17]:
identity_list[0:10]

[u'5A415A1E-3309-4554-9659-E2E1318103E3',
 u'606F6CD2-03C8-48BD-8C8F-244FA3BFBC32',
 u'2C9C579D-AA73-4B22-B16C-A298352EB8E0',
 u'9D6C2EAD-B548-47EA-B531-B1669E61EDD6',
 u'3CB74908-0CC0-465C-948E-C24BCEBB8627',
 u'59ACD8FD-BDA1-426C-B7DD-033F689BC0CE',
 u'E6A03C6C-DCCF-4EC2-AA95-4E8A3A2FF859',
 u'D5DC1FAB-037E-4C22-84C4-B0033BA5C8B7',
 u'CF7E2C09-B1F8-4D0D-BBF1-53A3DB939C2C',
 u'EB963525-A8B3-4F1A-86BF-0CD6F942D4BF']

In [44]:
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(text_list)

term_list = vectorizer.get_feature_names()

In [45]:
filter(match_number, text_list[4].split(' '))

[u'creation',
 u'of',
 u'shower',
 u'room',
 u'insid',
 u'of',
 u'exist',
 u'manufacturing/offic',
 u'build',
 u'(suit',
 u'near',
 u'washbay',
 u'-',
 u'center',
 u'of',
 u'suit',
 u'subject',
 u'to',
 u'field',
 u'inpsect']

In [46]:
buf = StringIO.StringIO()

In [47]:
for row, x in enumerate(X):
    dense_x = x.toarray().flatten()
    idx = np.where(dense_x != 0)[0]
    val = dense_x[idx]
    print >> buf, '%d' % category_list[row],
    for i, v in zip(*[idx, val]):
        print >> buf, ' %d:%f' % (i, v),
    print >> buf

In [48]:
contents = buf.getvalue()
f = open('seattle_tfidf.txt', 'w')
f.write(contents)
f.close()

In [49]:
term_list = vectorizer.get_feature_names()
f = codecs.open('seattle_terms.txt', 'w', 'utf-8')
for term in term_list:
    print >> f, '%s' % term
f.close()

In [50]:
f = codecs.open('seattle_identities.txt', 'w', 'utf-8')
for identity in identity_list:
    print >> f, '%s' % identity
f.close()

In [ ]:
import re

In [ ]:
pat = re.compile('\d+')

In [ ]:
def filter_out(pattern, words):
    update = []
    for x in words:
        if re.match(pattern, x) is None:
            update.append(x)
    return update

In [ ]:
filter_out(pat, ['hgahsga', '0hghgh', 'sds'])

In [ ]:
text = 'asa asas 1 jhjha 8278723'

In [ ]:
text.split(' ')

In [ ]:
print text
words = nltk.word_tokenize(text)
print words

In [ ]:
filter(f, words)


In [ ]:
def g(x):
    if re.match(pat, x) is None:
        return True

In [ ]:
print filter(g, words)

In [ ]:
type(re.match(pat, '4545'))

In [ ]:
re.match(pat, words[0])

In [ ]:
words

In [ ]:
'asasas as;lklaks asas 1212'.lower()